## Data Cleaning Notebook
Based on the LaLigaDataAnalysis notebook we will use this notebook to create a dataset to use in the quiniela game. We will do a table with the next features:

1. Seasons, division, matchday that will order the matchs (in this order).
2. A team column, for the team that plays at home.
3. A against column.
4. Columns W,L and T, that indicate the cummulative results of the team in that matchday.
5. A result column for that match.
6. A GD (gol difference) column for that match.
7. A cummulative gol difference column.
8. A raking column for the ranking position at this matchday
9. A column with the last 10 (if have it) results.
10. A column with the last 3 final rankings of last seasons


In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
with sqlite3.connect("../laliga.sqlite") as conn:
    df = pd.read_sql("SELECT * FROM Matches", con = conn)

df = df.loc[-df.score.isnull()]

In [3]:
def parse_score(score_as_string):
        if score_as_string is None: 
            return None
        home_goals, away_goals = score_as_string.split(":")
        home_goals = int(home_goals)
        away_goals = int(away_goals)
        if home_goals > away_goals: 
            return "1"
        if away_goals > home_goals: 
            return "2" 
        else: 
            return "X"

In [4]:
df['result'] = df['score'].apply(parse_score)

In [5]:
df[['home_goals','away_goals']] = df['score'].str.split(":",expand=True)
df = df.astype({"home_goals": int, "away_goals": int})

In [6]:
df['Difference_goals'] = abs(df['home_goals'] - df['away_goals'])

In [7]:
# We collect the information from the point of view of the home team and the away team, then we put all together
df3 = df[['season','division','home_team','result','home_goals','away_goals']]
SeasonStanding = df3.groupby(['season','division','home_team']).agg(
    GF_home = ('home_goals', sum),
    GA_home = ('away_goals', sum),
    W = ('result', lambda x: x.eq('1').sum()),
    L = ('result', lambda x: x.eq('2').sum()),
    T = ('result', lambda x: x.eq('X').sum())

).reset_index()
SeasonStanding.rename({'home_team':'team'}, axis=1, inplace=True)
SeasonStanding.sort_values(['season','division','team'])

df3_1 = df[["season","division","away_team","result","home_goals","away_goals"]]
df3_1 = df3_1.groupby(["season",'division',"away_team"], as_index=False).agg(
    GF_away = ("away_goals", sum),
    GA_away = ("home_goals", sum),
    W_away = ('result', lambda x: x.eq('2').sum()),
    L_away = ('result', lambda x: x.eq('1').sum()),
    T_away = ('result', lambda x: x.eq('X').sum())
)
df3_1.sort_values(['season','division','away_team'])

SeasonStanding['GF_away'] = df3_1['GF_away']
SeasonStanding['GA_away'] = df3_1['GA_away']
SeasonStanding['W_away'] = df3_1['W_away']
SeasonStanding['L_away'] = df3_1['L_away']
SeasonStanding['T_away'] = df3_1['T_away']

# Creating all the features
SeasonStanding['W'] = SeasonStanding['W'] + SeasonStanding['W_away']
SeasonStanding['L'] = SeasonStanding['L'] + SeasonStanding['L_away']
SeasonStanding['T'] = SeasonStanding['T'] + SeasonStanding['T_away']
SeasonStanding['GF'] = SeasonStanding['GF_home'] + SeasonStanding['GF_away']
SeasonStanding['GA'] = SeasonStanding['GA_home'] + SeasonStanding['GA_away']
SeasonStanding['GD'] = SeasonStanding['GF'] - SeasonStanding['GA']
SeasonStanding = SeasonStanding.drop(['GF_home', 'GF_away', 'GA_home', 'GA_away', 'W_away', 'L_away', 'T_away'], axis=1)
SeasonStanding['Pts'] = 3*SeasonStanding['W'] + SeasonStanding['T']

# Creating the rank for each season
team_count = SeasonStanding.groupby(['season','division'])['team'].count().tolist()
rank_column = []
for i in team_count :
    j = list(range(1,i+1,1))
    rank_column += j

SeasonStanding = SeasonStanding.sort_values(['season','division', 'Pts', 'GD', 'GF'], ascending=[True,True, False, False, False])
SeasonStanding['rank'] = rank_column

In [8]:
# Splitting season to compare it later
SeasonStanding['season2'] = SeasonStanding['season']
SeasonStanding = SeasonStanding.astype({"season": str})
SeasonStanding[['season','season2']] = SeasonStanding['season'].str.split("-",expand=True)
SeasonStanding = SeasonStanding.astype({"season": int})
SeasonStanding = SeasonStanding.drop('season2', axis=1)

In [9]:
def isNaN(obj):
    return obj != obj

In [10]:
def last_results(row, row_1, row_2, row_3, row_4, row_5, row_6, row_7, row_8, row_9, row_10):
    if isNaN(row_1):
        return [row]
    elif isNaN(row_2):
        return [row, row_1]
    elif isNaN(row_3):
        return [row, row_1, row_2]
    elif isNaN(row_4):
        return [row, row_1, row_2, row_3]
    elif isNaN(row_5):
        return [row, row_1, row_2, row_3, row_4]
    elif isNaN(row_6):
        return [row, row_1, row_2, row_3, row_4, row_5]
    elif isNaN(row_7):
        return [row, row_1, row_2, row_3, row_4, row_5, row_6]
    elif isNaN(row_8):
        return [row, row_1, row_2, row_3, row_4, row_5, row_6, row_7]
    elif isNaN(row_9):
        return [row, row_1, row_2, row_3, row_4, row_5, row_6, row_7, row_8]
    elif isNaN(row_10):
        return [row, row_1, row_2, row_3, row_4, row_5, row_6, row_7, row_8, row_9] 
    else:
        return [row,row_1,row_2,row_3,row_4, row_5, row_6, row_7, row_8, row_9, row_10]

In [11]:
def identity_col(col):
    return col

In [12]:
def result(row):
    if row['W']==1:
        return 'W'
    elif row['L']==1:
        return 'L'
    elif row['T']==1:
        return 'T'
    else:
        return 'Fail'

In [13]:
df4 = df[['season','division','home_team','result','matchday','home_goals','away_team','away_goals']]
MatchdayStanding = df4.groupby(['season','division','matchday','home_team']).agg(
    GF = ('home_goals', sum),
    Away_team = ('away_team',identity_col),
    GA = ('away_goals', sum),
    W = ('result', lambda x: x.eq('1').sum()),
    L = ('result', lambda x: x.eq('2').sum()),
    T = ('result', lambda x: x.eq('X').sum()),
).reset_index()
MatchdayStanding.rename({'home_team':'team'}, axis=1, inplace=True)
MatchdayStanding.sort_values(['season','division','team','matchday'])

df4_1 = df[["season","division","away_team","result",'matchday',"home_goals",'home_team',"away_goals"]]
df4_1 = df4_1.groupby(["season",'division','matchday',"away_team"], as_index=False).agg(
    GF = ("away_goals", sum),
    GA = ("home_goals", sum),
    Away_team = ('home_team',identity_col),
    W_away = ('result', lambda x: x.eq('2').sum()),
    L_away = ('result', lambda x: x.eq('1').sum()),
    T_away = ('result', lambda x: x.eq('X').sum())
)
df4_1.sort_values(['season','division','away_team','matchday'])
df4_1.rename({'away_team': 'team', 'W_away': 'W', 'L_away': 'L', 'T_away': 'T'}, axis=1, inplace=True) 

MatchdayStanding = MatchdayStanding.append(df4_1).sort_values(['season','division','matchday'])
MatchdayStanding.reset_index()
MatchdayStanding.sort_values(['season','division','matchday'])
MatchdayStanding.reset_index()
MatchdayStanding['result'] = MatchdayStanding.apply(lambda row: result(row), axis=1)


MatchdayStanding['GD'] = MatchdayStanding['GF'] - MatchdayStanding['GA']

# Cummulative results
MatchdayStanding['GF'] = MatchdayStanding.groupby(["season",'division','team'], as_index=False)['GF'].cumsum()
MatchdayStanding['GA'] = MatchdayStanding.groupby(["season",'division','team'], as_index=False)['GA'].cumsum()
MatchdayStanding['W'] = MatchdayStanding.groupby(["season",'division','team'], as_index=False)['W'].cumsum()
MatchdayStanding['L'] = MatchdayStanding.groupby(["season",'division','team'], as_index=False)['L'].cumsum()
MatchdayStanding['T'] = MatchdayStanding.groupby(["season",'division','team'], as_index=False)['T'].cumsum()


MatchdayStanding['GD_cum'] = MatchdayStanding['GF'] - MatchdayStanding['GA']
MatchdayStanding['Pts'] = 3*MatchdayStanding['W'] + MatchdayStanding['T']

# Raking by matchday
team_count = MatchdayStanding.groupby(['season','division','matchday'])['team'].count().tolist()
rank_column = []
for i in team_count :
    j = list(range(1,i+1,1))
    rank_column += j

MatchdayStanding = MatchdayStanding.sort_values(['season','division','matchday', 'Pts', 'GD', 'GF'], ascending=[True,True, True, False, False, False])
MatchdayStanding['rank'] = rank_column

# Shifted rows
MatchdayStanding = MatchdayStanding.sort_values(['season','division','team'], ascending=[True,True,True])
MatchdayStanding['result1'] = MatchdayStanding.groupby(['season','division','team'])['result'].shift()
MatchdayStanding['result2'] = MatchdayStanding.groupby(['season','division','team'])['result1'].shift()
MatchdayStanding['result3'] = MatchdayStanding.groupby(['season','division','team'])['result2'].shift()
MatchdayStanding['result4'] = MatchdayStanding.groupby(['season','division','team'])['result3'].shift()
MatchdayStanding['result5'] = MatchdayStanding.groupby(['season','division','team'])['result4'].shift()
MatchdayStanding['result6'] = MatchdayStanding.groupby(['season','division','team'])['result5'].shift()
MatchdayStanding['result7'] = MatchdayStanding.groupby(['season','division','team'])['result6'].shift()
MatchdayStanding['result8'] = MatchdayStanding.groupby(['season','division','team'])['result7'].shift()
MatchdayStanding['result9'] = MatchdayStanding.groupby(['season','division','team'])['result8'].shift()
MatchdayStanding['result10'] = MatchdayStanding.groupby(['season','division','team'])['result9'].shift()

MatchdayStanding['last_results'] = MatchdayStanding.apply(lambda row: last_results(row['result'],row['result1'],row['result2'],row['result3'],row['result4'],row['result5'],row['result6'],row['result7'],row['result8'],row['result9'],row['result10']), axis=1)

MatchdayStanding = MatchdayStanding.drop(['GF','Pts','GA','result1','result2','result3','result4','result5','result6','result7','result8','result9','result10'], axis=1)


In [14]:
# Splitting season to compare it later
MatchdayStanding['season2'] = MatchdayStanding['season']
MatchdayStanding = MatchdayStanding.astype({"season": str})
MatchdayStanding[['season','season2']] = MatchdayStanding['season'].str.split("-",expand=True)
MatchdayStanding = MatchdayStanding.astype({"season": int})
MatchdayStanding = MatchdayStanding.drop('season2', axis=1)

In [15]:
# Creating the column with the 5 last rankings
rank_col = []
for name, group in MatchdayStanding.groupby(['season','division','team','matchday','GD']):    
        concrete_rank = SeasonStanding['rank'].loc[(SeasonStanding['season'] < name[0]) & (SeasonStanding['team'] == name[2])]
        concrete_rank = concrete_rank[-5:]  # Only take care of last 5 seasons
        rank_col.append(concrete_rank.values)
        
MatchdayStanding.sort_values(['season','division','team','matchday','GD'])
MatchdayStanding['last_ranks'] = rank_col

In [22]:
# Creating the column with the 6 last rankings taking in count division
rank_col2 = []
concrete_rank = []
for name, group in MatchdayStanding.groupby(['season','division','team','matchday','GD']):
    # Taking in count last 5 seasons
    if(name[0] == 1929): season = name[0]-1
    elif(name[0] == 1930): season = name[0]-2
    elif(name[0] == 1929): season = name[0]-3
    elif(name[0] == 1929): season = name[0]-4
    else: season = name[0]-5
        
    while (season < name[0]):
        if(SeasonStanding['division'].loc[(SeasonStanding['season']==season) & (SeasonStanding['team']==name[2])].empty):
            actual_rank = 0
        elif((SeasonStanding['division'].loc[(SeasonStanding['season']==season) & (SeasonStanding['team']==name[2])] == 2).any()):
            actual_rank = SeasonStanding['rank'].loc[(SeasonStanding['season']==season) & (SeasonStanding['team']==name[2])] + \
            SeasonStanding['team'].loc[(SeasonStanding['season']==season) & (SeasonStanding['division']==1)].nunique()
        else:
            actual_rank = SeasonStanding['rank'].loc[(SeasonStanding['season']==season) & (SeasonStanding['team']==name[2])]
        concrete_rank.append(actual_rank)
        season += 1
    rank_col.append(concrete_rank)
        

In [23]:
MatchdayStanding.sort_values(['season','division','team','matchday','GD'])
MatchdayStanding['last_ranks_total'] = rank_col

ValueError: Length of values (215857) does not match length of index (95999)

In [19]:
MatchdayStanding

,season,division,matchday,team,Away_team,W,L,T,result,GD,GD_cum,rank,last_results,last_ranks
0,1928,1,1,Arenas Club,Athletic Madrid,0,1,0,L,-1,-1,7,[L],[]
5,1928,1,2,Arenas Club,Catalunya,0,2,0,L,-3,-4,10,"[L, L]",[]
10,1928,1,3,Arenas Club,Real Unión,0,2,1,T,0,-4,9,"[T, L, L]",[]
15,1928,1,4,Arenas Club,Barcelona,0,2,2,T,0,-4,9,"[T, T, L, L]",[]
20,1928,1,5,Arenas Club,Athletic,1,2,2,W,1,-3,6,"[W, T, T, L, L]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47989,2021,2,2,UD Ibiza,Málaga CF,0,0,2,T,0,0,12,"[T, T]",[]
47998,2021,2,3,UD Ibiza,CD Leganés,1,0,2,W,1,1,6,"[W, T, T]",[]
47978,2021,2,1,UD Las Palmas,Real Valladolid,0,0,1,T,0,0,10,[T],"[11, 18, 12, 14, 19]"
47988,2021,2,2,UD Las Palmas,Girona,0,0,2,T,0,0,14,"[T, T]","[11, 18, 12, 14, 19]"


In [18]:
import openpyxl

# Save as an excel file
MatchdayStanding.to_excel(r'..\reports\CleanedData.xlsx', header=True)